In [1]:
import operator, re, os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from neurotools.io.files import load_embedding_model
from neurotools.io.gdrive import download_sheets_doc
from neurotools.plot import figure
from sklearn.feature_extraction.text import TfidfVectorizer
from neurotools.language.simple_tokenizer import SimpleTokenizer
from wordcloud import WordCloud
from googletrans import Translator
import itertools
import ast
from neurotools.io.gdrive import download_sheets_doc
%matplotlib inline

In [2]:
def get_average_vector(words, emb):
    """
        Average the word vectors of a list of words.
    """
    words_in_emb = [emb[word] for word in words if word in emb.vocab.keys() and word not in ['bezpieczeństwo', 'pistolet']]
    return np.mean(words_in_emb, axis=0)

In [3]:
def get_idf_map(documents):
    tfidf = TfidfVectorizer(use_idf=True)
    tfidf.fit_transform(documents)
    word_to_idf = dict((x[0], tfidf.idf_[x[1]]) for x in sorted(tfidf.vocabulary_.items(), key=operator.itemgetter(1)))
    return word_to_idf

In [4]:
def get_idf_weighted_average_vector(words, word_to_idf, emb):
    """
        Average the idf-weighted word vectors of a list of words. (Map this function to every document)
    """
    # only consider words IN embedding model and in TFIDF vectorizer
    words = [word for word in words if word in word_to_idf.keys() and word in emb.vocab.keys() and word not in ['bezpieczeństwo', 'pistolet']]
    weighted = [word_to_idf[word] * emb[word] for word in words]
    return np.mean(weighted, axis=0)

In [5]:
def cosine_similarity(a,b):
    """
        Compute the cosine similarity of a and b.
    """
    return np.dot(a,b) / ( (np.dot(a,a) **.5) * (np.dot(b,b) ** .5) )

In [6]:
def get_similarity_with_dimensions(row, dimensions):
    """
        Get similarity of row's word vector and every dimension.
        row : row of the movie dataframe
        dimensions : dataframe containing Neuroflash dimensions
    """
    for k in range(dimensions['dimension'].shape[0]):
        name = str(dimensions['en_label'].iloc[k])
        # define a new column for the cosine similarity of dimension k and the post text
        row[name] = cosine_similarity(row['wv'], dimensions['dimension'].iloc[k])
    return row

# fix dimensions csv

In [7]:
tokenizer = SimpleTokenizer('pl') 

Extra characters read for pl
Stopwords set read for pl
Lemma lookup read for pl


In [8]:
emb, word2rank_dict = load_embedding_model('../utils/pl_pl_commoncrawl_v3.bin')

In [38]:
dimensions = pd.read_csv('../utils/dimensions.csv')

In [11]:
existing = set(dimensions.en_label)

In [14]:
dimension_sheet_id = '1qa6hmUwjipTocTLIAIBEuObkzVGwWUvYXV04HjMw0BM'
dimension_sheet = download_sheets_doc(dimension_sheet_id, has_column_labels=True)['dimensions_v2.csv']

In [15]:
from_dimension_sheet = dimension_sheet[dimension_sheet['vw_positive_relevant']=='1'][['en_label', 'en', 'pl']]
from_dimension_sheet['en_label'] = from_dimension_sheet['en_label'].str.title()

In [16]:
to_check = set(from_dimension_sheet.en_label)

In [23]:
to_add_dims = to_check - existing

In [24]:
to_add = from_dimension_sheet.loc[from_dimension_sheet['en_label'].isin(to_add_dims)]

In [28]:
to_add['tokens'] = to_add['pl'].map(lambda words: tokenizer.tokenize(' '.join([word for word in words.split() if not word.startswith('nie')])))

/Users/piocalderon/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
to_add.head()

,en_label,en,pl,tokens
217,Support,enables support help,pomagać wsparcie łagodzić upoważniać,"[pomagać, wsparcie, łagodzić, upoważniać]"
218,Inspire,aplify inspire generate trigger ignite,inspirować zachęcać motywować napędzać,"[inspirować, zachęcać, motywować, napędzać]"
219,Happiness,,miłość szczęście radość szczęśliwy,"[miłość, szczęście, radość, szczęśliwy]"
220,Celebrate,,Świętować obchodzić celebrować urodziny święto...,"[świętować, obchodzić, celebrować, urodziny, ś..."
221,Meaningful,,doniosły istotny wybitny znaczący znakomity,"[donieść, istotny, wybitny, znaczyć, znakomity]"


In [33]:
to_remove = existing - to_check

In [9]:
to_remove = ['Being (to be)_project_sheet',
 'Celebrate_project_sheet',
 'Comfort_project_sheet',
 'Equitable_project_sheet',
 'Happiness_project_sheet',
 'Innovative_project_sheet',
 'Inspire_project_sheet',
 'Meaningful_project_sheet',
 'Reliable_project_sheet',
 'Responsible_project_sheet',
 'Safety_project_sheet',
 'Support_project_sheet',
 'Valuable_project_sheet',
 'Worth_the_money_project_sheet']

In [10]:
dimensions = dimensions.loc[~dimensions['en_label'].isin(to_remove)]

NameError: name 'dimensions' is not defined

In [13]:
dimensions.loc[72,'en_label'] = 'Feel Good'

NameError: name 'dimensions' is not defined

In [14]:
dimensions['tokens'] = dimensions['tokens'].map(ast.literal_eval)

NameError: name 'dimensions' is not defined

In [48]:
concated = pd.concat([dimensions, to_add], axis=0, ignore_index = True)
concated.to_csv('../utils/dimensions2.csv', index=False)

In [50]:
concated[['en_label', 'en', 'pl']].to_csv('../utils/gordian_pick.csv', index=False)

In [114]:
to_add['dimension'] = to_add['tokens'].map(lambda x: get_average_vector(x, emb))

/Users/piocalderon/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [121]:
concated['dimension'] = concated['tokens'].map(lambda x: get_average_vector(x, emb))

In [122]:
concated.loc[49]

en_label                                             Lifestyle
en                                    life, lifestyle, freedom
pl                                  życie, styl_życia, wolność
tokens                           [żyto, stylo, życie, wolność]
dimension    [-2.6182234, 0.64857113, 2.1783419, 0.3793544,...
Name: 49, dtype: object

# fix processed data

In [11]:
files = os.listdir('../processed')

In [12]:
for file in files:
    print(file)
    dat = pd.read_csv('../processed/' + file)
    remove_cols = [x for x in dat.columns if 'project_sheet' in x.lower() and 'Feel Good' not in x]
    dat = dat.rename({'Feel Good_project_sheet': 'Feel Good'}, axis=1)
    dat = dat[[x for x in dat.columns if x not in remove_cols]]
    dat['tokens'] = dat['tokens'].map(ast.literal_eval)
    fb_word_to_idf = get_idf_map(dat['tokens'].map(lambda x: ' '.join(x)).values)
    dat['wv'] = dat['tokens'].map(lambda x: get_idf_weighted_average_vector(x, fb_word_to_idf, emb))
    dat = dat.apply(lambda x: get_similarity_with_dimensions(x, to_add), axis=1)
    dat.to_csv('../processed/' + file, index=False)

processed_citroenpolska.csv


/Users/piocalderon/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in multiply
  import sys


NameError: ("name 'to_add' is not defined", 'occurred at index 0')

In [4]:
dat = pd.read_csv('../processed/' + files[0])

In [28]:
.index

Index(['Bargain', 'Cool', 'Natural'], dtype='object')

In [31]:
statistics = []
for file in files:
    if file.startswith('processed'):
        print(file)
        dat = pd.read_csv('../processed/' + file)
        top3_names = dat[[x for x in dat.columns if x not in ['Total_likes', 'Shares_count', 'Total_comments', 'Post_utime']]].corr()['success'].sort_values(ascending=False).iloc[1:4].index
        top3_scores = dat[[x for x in dat.columns if x not in ['Total_likes', 'Shares_count', 'Total_comments', 'Post_utime']]].corr()['success'].sort_values(ascending=False).iloc[1:4].values

        innovative = dat[dat['Innovative'] > 0.3]
        innovative_names = innovative[[x for x in innovative.columns if x not in ['Total_likes', 'Shares_count', 'Total_comments', 'Post_utime', 'Innovative']]].corr()['success'].sort_values(ascending=False).iloc[1:4].index
        innovative_scores = innovative[[x for x in innovative.columns if x not in ['Total_likes', 'Shares_count', 'Total_comments', 'Post_utime', 'Innovative']]].corr()['success'].sort_values(ascending=False).iloc[1:4].values
        
        good = dat[dat['Feel Good'] > 0.3]
        good_names = dat[[x for x in dat.columns if x not in ['Total_likes', 'Shares_count', 'Total_comments', 'Post_utime', 'Feel Good']]].corr()['success'].sort_values(ascending=False).iloc[1:4].index
        good_scores = dat[[x for x in dat.columns if x not in ['Total_likes', 'Shares_count', 'Total_comments', 'Post_utime', 'Feel Good']]].corr()['success'].sort_values(ascending=False).iloc[1:4].values
       
        information = {
            'brand': file[10:-4],
            'overall_top1': top3_names[0],
            'overall_top2': top3_names[1],
            'overall_top3': top3_names[2],
            'overall_score1': top3_scores[0],
            'overall_score2': top3_scores[1],
            'overall_score3': top3_scores[2],
            'innovative_top1': innovative_names[0],
            'innovative_top2': innovative_names[1],
            'innovative_top3': innovative_names[2],
            'innovative_score1': innovative_scores[0],
            'innovative_score2': innovative_scores[1],
            'innovative_score3': innovative_scores[2],
            'good_top1': good_names[0],
            'good_top2': good_names[1],
            'good_top3': good_names[2],
            'good_score1': good_scores[0],
            'good_score2': good_scores[1],
            'good_score3': good_scores[2],            
        }
        statistics.append(information)
pd.DataFrame(statistics)[['brand', 'overall_top1', 'overall_score1', 'overall_top2', 'overall_score2', 'overall_top3', 'overall_score3', 'innovative_top1', 'innovative_score1', 'innovative_top2', 'innovative_score2', 'innovative_top3', 'innovative_score3', 'good_top1', 'good_score1', 'good_top2', 'good_score2', 'good_top3', 'good_score3']].to_csv('../utils/statistics_dims.csv', index=False)

processed_citroenpolska.csv
processed_ford.polska.csv
processed_mercedesbenzpolska.csv
processed_peugeotpolska.csv
processed_volkswagenpolska.csv
processed_renaultpolska.csv
processed_hondapolska.csv
processed_ferrarikatowice.csv
processed_bmw.polska.csv
processed_seatpolska.csv
processed_landroverpolska.csv
processed_hyundaipolska.csv
processed_skodapolska.csv
processed_jaguarpolska.csv
processed_toyotapolska.csv
processed_kiamotorspolska.csv
processed_lexuspolska.csv
processed_fiatpl.csv
processed_jeeppl.csv
processed_opelklub.csv
processed_audipl.csv
processed_porschepl.csv


In [38]:
dat['Post_type'].value_counts()

photo     1717
video      223
link        62
status      18
Name: Post_type, dtype: int64

In [39]:
statistics = []
for file in files:
    if file.startswith('processed'):
        print(file)
        dat = pd.read_csv('../processed/' + file)
        counts = dat['Post_type'].value_counts()
        cont = dat['Post_type'].value_counts().index
        
        photo = 0
        video = 0
        link = 0
        status = 0
        if 'photo' in cont:
            photo = counts['photo']
        if 'video' in cont:
            video = counts['video']
        if 'link' in cont:
            link = counts['link']
        if 'status' in cont:
            status = counts['status']        
        
        information = {
            'brand': file[10:-4],
            'photos': photo,
            'videos': video,
            'links': link,
            'status': status
        }
        statistics.append(information)
pd.DataFrame(statistics).to_csv('../utils/statistics_type.csv', index=False)

processed_citroenpolska.csv
processed_ford.polska.csv
processed_mercedesbenzpolska.csv
processed_peugeotpolska.csv
processed_volkswagenpolska.csv
processed_renaultpolska.csv
processed_hondapolska.csv
processed_ferrarikatowice.csv
processed_bmw.polska.csv
processed_seatpolska.csv
processed_landroverpolska.csv
processed_hyundaipolska.csv
processed_skodapolska.csv
processed_jaguarpolska.csv
processed_toyotapolska.csv
processed_kiamotorspolska.csv
processed_lexuspolska.csv
processed_fiatpl.csv
processed_jeeppl.csv
processed_opelklub.csv
processed_audipl.csv
processed_porschepl.csv


In [11]:
statistics = []
for file in files:
    if file.startswith('processed'):
        print(file)
        dat = pd.read_csv('../processed/' + file)
        information = {
            'brand': file[10:-4],
            'start_date': pd.to_datetime(dat['Post_utime'],unit='s').min(),
            'end_date': pd.to_datetime(dat['Post_utime'],unit='s').max(),
            'num_rows': dat.shape[0],
            'likes': dat['Total_likes'].sum(),
            'comments': dat['Total_comments'].sum(),
            'shares': dat['Shares_count'].sum()    
        }
        statistics.append(information)
pd.DataFrame(statistics).to_csv('../utils/statistics_success.csv', index=False)

processed_citroenpolska.csv
processed_ford.polska.csv
processed_mercedesbenzpolska.csv
processed_peugeotpolska.csv
processed_volkswagenpolska.csv
processed_renaultpolska.csv
processed_hondapolska.csv
processed_ferrarikatowice.csv
processed_bmw.polska.csv
processed_seatpolska.csv
processed_landroverpolska.csv
processed_hyundaipolska.csv
processed_skodapolska.csv
processed_jaguarpolska.csv
processed_toyotapolska.csv
processed_kiamotorspolska.csv
processed_lexuspolska.csv
processed_fiatpl.csv
processed_jeeppl.csv
processed_opelklub.csv
processed_audipl.csv
processed_porschepl.csv


In [2]:
statistics = []
for file in files:
    print(file)
    dat = pd.read_csv('../processed/' + file)
    information = {
        'brand': file[10:-4],
        'start_date': pd.to_datetime(dat['Post_utime'],unit='s').min(),
        'end_date': pd.to_datetime(dat['Post_utime'],unit='s').max(),
        'num_rows': dat.shape[0],
        'good_25': dat[dat['Feel Good'] > 0.25].shape[0],
        'good_30': dat[dat['Feel Good'] > 0.3].shape[0],
        'good_40': dat[dat['Feel Good'] > 0.4].shape[0],
        'innovative_25': dat[dat['Innovative'] > 0.25].shape[0],
        'innovative_30': dat[dat['Innovative'] > 0.3].shape[0],
        'innovative_40': dat[dat['Innovative'] > 0.4].shape[0]        
    }
    statistics.append(information)
pd.DataFrame(statistics).to_csv('../utils/statistics.csv', index=False)

NameError: name 'files' is not defined

In [140]:
pd.to_datetime(dat['Post_utime'],unit='s').max()

Timestamp('2017-12-05 18:00:00')

,Post,Post_utime,Post_type,Post_text,Total_likes,Total_comments,Shares_count,tokens,success,wv,...,Safety,Teaching,Technical,Valuable,Feel Good,Support,Inspire,Happiness,Celebrate,Meaningful
0,https://www.facebook.com/165622486798242_33779...,1325540959,video,"Ten """"lekko dłubnięty"""" Jeep JK naprawdę robi ...",7,1,1,"[lekko, dłubnięty, jeep, jk, naprawdę, robić, ...",11,"[-9.023091, -3.11451, -6.257595, -1.1175709, 1...",...,0.193496,0.197620,0.021515,0.213790,0.427162,0.076468,0.183659,0.306481,0.048041,0.231278
1,https://www.facebook.com/165622486798242_12119...,1325503611,video,Kibicujemy !!! DAKAR 2012 - Jeep Wrangler Rubi...,2,1,1,"[kibicować, dakar, jeep, wrangler, rubicon, nu...",6,"[-0.12943335, -21.169708, -11.048273, 5.149941...",...,-0.117024,-0.046474,-0.148884,-0.155240,-0.044552,-0.149972,-0.025723,0.014376,0.082779,-0.112518
2,https://www.facebook.com/165622486798242_26724...,1325591721,video,Kolejny test zaliczony na 6 !,7,1,2,"[kolejny, testo, zaliczyć]",13,"[2.261546, 15.728859, -7.5990906, 10.215474, 7...",...,0.177143,0.221931,0.061032,0.174944,0.134249,0.042103,0.068086,0.088806,0.085125,0.180768
3,https://www.facebook.com/165622486798242_16271...,1325591465,video,Porównania w Rodzinie bez kompleksów. Ojciec I...,3,1,0,"[porównanie, wiek, rodzina, beza, kompleks, oj...",5,"[-10.116509, 4.788958, 0.014097023, 9.202602, ...",...,0.202132,0.208578,-0.028238,0.163740,0.221521,0.137073,0.049854,0.370126,0.150098,0.256525
4,https://www.facebook.com/165622486798242_33222...,1325693740,link,Specjaliści z Hennessey Performance wycisnęli ...,14,4,4,"[specjalista, z, hennessey, performance, wycis...",30,"[-6.02795, 2.0940857, -7.533383, 2.665072, 8.9...",...,0.140862,0.121712,0.167850,0.167282,0.140372,0.127072,0.142965,0.171209,-0.019286,0.166786
5,https://www.facebook.com/165622486798242_34882...,1325669772,photo,WYPRZEDAŻ ROCZNIKA 2011. Spiesz się zanim ktoś...,8,9,6,"[wyprzedaż, rocznik, spieszyć, siebie, zanim, ...",38,"[-2.9706688, -2.5330331, 1.6146855, -3.150104,...",...,0.127043,0.103981,-0.092230,0.251648,0.233816,-0.038928,0.013331,0.205172,0.125879,0.088055
6,https://www.facebook.com/165622486798242_15925...,1325782787,video,Jeep każdego zmienia na lepsze :o),10,1,2,"[jeep, zmieniać, lepszy, ojciec]",16,"[-1.0644859, 15.916132, -6.1366205, -0.8600403...",...,0.242623,0.283073,0.140041,0.254251,0.282862,0.193939,0.285472,0.219076,0.073137,0.360476
7,https://www.facebook.com/165622486798242_26881...,1325781240,video,Jeep - Parkujesz gdzie chcesz.,14,2,3,"[jeep, parkować, chcieć]",24,"[-4.430994, 7.1857686, 7.6168733, -7.302443, -...",...,0.224101,0.147445,-0.035033,0.226545,0.168427,0.118218,0.087386,0.133632,0.039732,0.106935
8,https://www.facebook.com/165622486798242_35036...,1325852007,photo,Takie tam... Przy śniadaniu ;o),15,1,7,"[tama, przy, śniadanie, ojciec]",31,"[-17.131645, 6.5623107, -5.9790735, 1.2777019,...",...,0.193195,0.182259,-0.041315,0.257109,0.299773,0.164994,0.185895,0.299822,0.153959,0.170233
9,https://www.facebook.com/165622486798242_14859...,1325934002,video,Jeszcze malutki a już taki dzielny ;o),7,2,1,"[jeszcze, malutki, a, taka, dzielny, ojciec]",13,"[-19.594833, -1.0420078, -0.77450734, 1.734857...",...,0.270989,0.190691,-0.138921,0.330775,0.526103,0.135262,0.144688,0.477900,0.201486,0.300136


In [34]:
dats = pd.read_csv('../processed/processed_mercedesbenzpolska.csv')

In [43]:
dats.dropna()

,Post,Post_utime,Post_type,Post_text,Total_likes,Total_comments,Shares_count,tokens,success,wv,...,Safety,Teaching,Technical,Valuable,Feel Good,Support,Inspire,Happiness,Celebrate,Meaningful
0,https://www.facebook.com/165245993503617_32241...,1325521152,photo,Pierwszy w historii roadster z niemal całkowic...,110,22,16,"['pierwszy', 'wiek', 'historii', 'roadster', '...",186,[ -4.6637216 7.4148417 0.14112116 -0.47...,...,0.178806,0.055394,0.206482,0.167174,0.084766,0.050190,0.117690,0.075606,-0.076139,0.162053
1,https://www.facebook.com/165245993503617_18473...,1325579939,link,Czy wśród Waszych noworocznych życzeń są też t...,87,50,6,"['cza', 'wśród', 'noworoczny', 'życzyć', 'tenż...",199,[-10.389376 13.973338 -1.3835561 5.07...,...,0.282728,0.307991,-0.079172,0.448052,0.472081,0.214011,0.391928,0.361028,0.311720,0.405396
2,https://www.facebook.com/165245993503617_13839...,1325692704,photo,18% niższy współczynnik oporu i 5% mniejsze zu...,58,9,13,"['niższy', 'współczynnik', 'opór', 'i', 'mniej...",102,[ -7.412585 6.703234 -3.7146635 7.66...,...,0.250175,0.191032,0.331055,0.267320,0.192784,0.206182,0.304892,0.131890,-0.023069,0.288315
3,https://www.facebook.com/165245993503617_34015...,1325861633,video,Już w niedzielę startuje 20 finał WOŚP. Zachęc...,52,14,6,"['wiek', 'startować', 'finał', 'wośp', 'zachęc...",92,[ -6.1367173 10.869985 1.326364 7.65...,...,0.368130,0.367709,0.021159,0.344967,0.450161,0.378422,0.444331,0.361608,0.411345,0.438263
4,https://www.facebook.com/165245993503617_14775...,1326125105,link,"Czy wiecie, że nowy roadster SL waży aż 125 kg...",51,5,7,"['cza', 'wiedzieć', 'nowy', 'roadster', 'sl', ...",75,[ -7.897668 10.246725 -4.867039 1.43...,...,0.277044,0.236484,0.307856,0.356318,0.282341,0.183311,0.215917,0.193508,-0.015433,0.404373
5,https://www.facebook.com/165245993503617_34195...,1326107647,status,Już tylko do jutra trwa nasz noworoczny konkur...,30,12,2,"['tylko', 'jutro', 'trwać', 'noworoczny', 'kon...",58,[ -7.6947494 9.401981 2.333932 -0.53...,...,0.207185,0.197192,-0.143622,0.329485,0.309384,0.137598,0.226562,0.278305,0.274096,0.225337
6,https://www.facebook.com/165245993503617_18853...,1326297149,video,Niedawno pytaliśmy o zeszłoroczne wydarzenia d...,54,13,4,"['pytać', 'ojciec', 'zeszłoroczny', 'wydarzyć'...",88,[-1.09560232e+01 1.40525160e+01 -2.36851311e+...,...,0.298435,0.392479,0.061055,0.464437,0.443984,0.185503,0.340796,0.370645,0.261398,0.522909
7,https://www.facebook.com/165245993503617_19079...,1326470834,photo,Dynamiczna hybrydowa limuzyna o wysokim poziom...,72,6,8,"['dynamiczny', 'hybrydowy', 'limuzyna', 'ojcie...",100,[-5.67050314e+00 6.16678190e+00 -3.70130062e+...,...,0.171414,0.209393,0.262600,0.244340,0.249158,0.146840,0.235264,0.154693,0.076646,0.333512
8,https://www.facebook.com/165245993503617_27658...,1326732022,photo,"Czy wiecie, że odśnieżaniem pasów startowych, ...",41,11,3,"['cza', 'wiedzieć', 'odśnieżać', 'pasy', 'star...",69,[-11.540943 2.485166 4.3659844 -0.74...,...,0.348120,0.210799,0.224983,0.316654,0.281814,0.243313,0.240216,0.237429,0.084141,0.233652
9,https://www.facebook.com/165245993503617_34687...,1326717726,status,Chcielibyśmy podziękować Wam za wszystkie zgło...,22,6,0,"['chcieć', 'podziękować', 'za', 'wszystek', 'z...",34,[-12.67648 13.202057 3.0975695 -5.04...,...,0.333496,0.235985,-0.117050,0.400937,0.459870,0.216154,0.274192,0.402632,0.220847,0.313187


In [35]:
dats[[x for x in dats.columns if x not in ['Post_utime', 'wv', 'tokens', 'Post', 'Total_likes', 'Total_comments', 'Shares_count']]].to_csv('../utils/mercedes.csv', index=False)

In [27]:
dats[[x for x in dats.columns if x not in ['Post_utime', 'Total_likes', 'Total_comments', 'Shares_count']]].corr()['success']

success          1.000000
Trendy          -0.212727
Kind            -0.288048
Polite          -0.255758
Warm            -0.277782
Diligent        -0.299193
Blue_Collar     -0.205347
White_Collar    -0.237195
Protective      -0.270652
Serious         -0.227264
Funny           -0.093062
Pleasant        -0.192822
Beautiful       -0.152563
Alive           -0.293224
Above           -0.240340
Descriptive     -0.215734
Cheap           -0.183353
Expensive       -0.279743
Big             -0.276425
Small           -0.245554
Modern          -0.299190
Traditional     -0.181143
Appealing       -0.257680
Sustainable     -0.303161
Practical       -0.275989
Active          -0.276361
Easy            -0.223121
Trustworthy     -0.309902
Natural         -0.199722
Arts            -0.108361
                   ...   
Science         -0.232869
Open            -0.246562
Cool            -0.164727
Likeable        -0.229105
Children        -0.177043
Community       -0.292733
Culture         -0.241079
Design      